# Notebook pour l'implémentation d'une première pipeline

Le but de ce notebook est de faire une première pipeline qui, à partir d'un ensemble typique de documents, génère la demande de financements souhaitée.

## Load documents

In [ ]:
# Here, load the documents in python
# Function to read the questions inside files with .docx extension contained in a folder
def Read_Questions_From_docx (PathFolderSource, PathForOutputsAndLogs):
# This program reads the content of files with .docx extension contained in a folder
# It uses python-docx 1.1.2 to manipulate Word documents : .docx only but not .doc so you need first to type "pip install python-docx" in your terminal
# It identifies the questions for AI from the other information by looking for the tag "SQAI " at the beginning of the question
# and for the tag "EQAI" at the end of the question (a question can have several paragraphs)
# SQAI stands for Start of the Question for AI and EQAI stands for End of the Question for AI
# The ouptput of this function is double :
# 1°) return a dictionary containing the questions for AI : Key= "NameOfFile - Question Number" and Value = Text of the question
# 2°) create in a folder a new version of each document that has been read, where below each question,
#  is added the same Key "NameOfFile - Question Number"
# After the answers are created, It will allow to insert the answers at the right place just below the corresponding question in the documents
# The user will then be able to see and modify in each document the original question and the answer given by the AI
# The function also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"


    #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

    ItIsAQuestion = False # Tag that indicates if the current paragraph is inside a question for AI
    QuestionNumber = 0 # Number of the question for AI
    TheTextofTheQuestion = '' # Text of a question for AI
    DictQuestions = {} #initialise the dictionnary of questions
    # read content of the files, only if they are .docx (extension to other file types possible with the match - case)
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary
                    for docpara in document.paragraphs:
                        if (docpara.text != ''): # we don't want to add empty paragraphs
                            if(docpara.text[:4]=='SQAI'): # if first characters are "SQAI", then it is the start of a question for AI
                                ItIsAQuestion = True
                                TheTextofTheQuestion = docpara.text[4:]# eliminate the 4 first characters which are the TAG "SQAI"
                            else:
                                if (ItIsAQuestion): # if we are inside a question
                                    TheTextofTheQuestion = TheTextofTheQuestion + ". "+ docpara.text
                            if (docpara.text[-4:]=='EQAI'): # if the end of the paragraph is EQAI, then it is the end of the question
                                ItIsAQuestion = False
                                TheTextofTheQuestion = TheTextofTheQuestion[:-4]# eliminate the 4 last characters which are the TAG "EQAI"
                                QuestionNumber += 1
                                QuestionUI = NameOfDocument + ' - ' + str(QuestionNumber)
                                DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                docpara.text = docpara.text + '\n' + QuestionUI
                                #TO DO AFTER : manager les infos entre les questions si on doit les fournir à l'IA
                                #TO DO AFTER : dans un dictionaire de complément d'infos
                                #TO DO AFTER : Gérer les numérotations indentées qui sous-divisent les questions ?
                                #TO DO AFTER : Gérer les tableaux ?
                                #TO DO AFTER : Gérer la résistance à l'erreur = début SQAI mais manque fin EQAI ou inverse

                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument)
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

            case '.doc':
                print('Fichier DOC')# OPEN QUESTION: do we consider reading .doc files ?
            case _:
                print('Fichier non pris en charge')
                #OPEN QUESTION: do we consider reading other types of files below ?
                #'rtf', 'pdf', 'xls', 'xlsx', 'csv', 'ppt', 'pptx',
                #'odc','odf', 'odg', 'odm', 'odp', 'ods','odt', 'odx'
                # WE SHOULD CHECK ALL EXTENSIONS OF THE FILES CONTAINED IN THE FOLDER 
                # AND PROMPT A MESSAGE IF EXTENSION NOT MANAGED
    print('End of the read program')
    return DictQuestions

# Function to write the answer below each question inside files with .docx extension contained in a folder
def Write_Answers_in_docx (PathFolderSource, DictonaryOfAnswers, PathForOutputsAndLogs):
# The main program has already submitted each question to the AI 
# and filled the "DictonaryOfAnswers" with the answers to the questions 
# The "DictonaryOfAnswers" has the same Key "NameOfFile - Question Number" as the "DictonaryOfQuestions"
# Then the main program will call the "Write_Answers_in_docx" function to write the answers 
# from the he "DictonaryOfAnswers" into the documents themselves
# As the read function has already placed the key of the question below the question, 
# this function will just have to find the key below the question and replace ti by the answer, back in the docx file 
# It will also remove the SQAI and EQAI tags from the questions


     #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)
    #FilesWithPath.remove(PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt') # remove the log file from the list of files to be read
    #TO DO AFTER : manage the case where the log file is not in the folder
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary

                    # for each key of the dictionary, corresponding to the document
                    # find the key in the document and replace it by the answer
                    # As the key was below the question, this puts the answer just below the question
                    # if the key is not found, log an error

                    # Create a subset of the dictionary corresponding to the document opened
                    Dict_Of_Answers_of_the_Document = dict(filter(lambda item: item[0].split(' - ')[0] == NameOfDocument, DictonaryOfAnswers.items()))
                    print(Dict_Of_Answers_of_the_Document) # The answer dictionnary for the document
                    for docpara in document.paragraphs:
                        for key, value in Dict_Of_Answers_of_the_Document.items():
                            if key in docpara.text:
                                docpara.text = docpara.text.replace(key, value)
                                # Dict_Of_Answers_of_the_Document.pop(key) # remove the key from the dictionnary when it has been found

                    # Now, we suppress the tags SQAI and EQAI from the questions
                    for docpara in document.paragraphs:
                        if "SQAI" in docpara.text:
                            docpara.text = docpara.text.replace("SQAI", "")
                        if "EQAI" in docpara.text:
                            docpara.text = docpara.text.replace("EQAI", "")

                    # We create a new version of the document with the answers
                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument[:-4] + "_with_answers.docx")
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when opening for writing the Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

    print('End of the write program')
    return



# *******************************MAIN PROGRAM*************************************
# Settings for the path files
Path_where_we_put_Outputs = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python/Dev/Dev IA Asso/Pour les logs/' 
Folder_where_the_files_are = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python/Dev/Dev IA Asso/LesFilesA Lire/'
from docx import Document # import de python-docx

# Read the questions in the files and put them into a dictionnary
The_Dict_Of_Questions = Read_Questions_From_docx (Folder_where_the_files_are, Path_where_we_put_Outputs)

# TO DO : The main programm should then call the AI to answer the questions of the dictionary "The_Dict_Of_Questions"
# and put the answers into a "dictionnary of answers" with the same keys (key of question = key of answer)

# For the moment, we create a dictionary of answers with the same keys as the dictionary of questions
# by just taking the question as the answer we just put "ANSWER TO: " + the question

for key, value in The_Dict_Of_Questions.items():
        The_Dict_Of_Answers = {key:  value for key,  value in The_Dict_Of_Questions.items()}
for key, value in The_Dict_Of_Answers.items():
        The_Dict_Of_Answers[key] = '\rn' + 'ANSWER TO: ' + value
# Write the answers into the docx files just below the questions
Write_Answers_in_docx (Path_where_we_put_Outputs, The_Dict_Of_Answers, Path_where_we_put_Outputs)

#********************************END OF MAIN PROGRAM********************************


## (Optional in the beginning) Chunk and embedd documents

Chunking and embedding documents is a way to implement a RAG (Retrieval Augmented Generation). 

To learn about this concept, you can check the following links :

Here are also useful resources to implement a RAG in python using langchain :



!! It is important to note that while RAG is a common way to provide LLMs with context, specific methods can be used for this project. For instance, maybe that all documents have an "information about x" section that can be directly retrieved with regex methods to provide the model with.

For regex methods, you can find documentation here :


In [ ]:
# Here split the document into chunks

In [ ]:
# Here embed those chunks

In [ ]:
# (Optional) Here you can store those embedded chunks into a vector store

## call a large language model via an API (e.g. Mistral API call - use free tiers)

Here we're gonna call a model (and pass him the context if already implemented before)

Some links you can check to learn more if you don't know how it works :

Langchain (one of the classic tools for this kind of task)


<b>To run a model locally</b>

With Ollama :

With huggingface : 

In [ ]:
"""
Here, first write your credentials for API call (don't push it on git !! Use environment variables)
or load the model in the notebook kernel if you want to use a model locally
"""

In [ ]:
"""
Then, implement API calling (langchain chain + prompt engineering)
You can divide the whole process in several sub-questions if the model can't take enough context at once,
or if it does not perform well enough.
"""

## (Very very optional) Implement a langgraph to enhance generation performances with agentic behavior

This step should not be necessary but once everything else is set up, you can play with it.

Documentation : 

In [ ]:
# Langgraph implementation